In [1]:
# Set covering problem: you need to select a certain number of blocks in order to cover all the initial sets
# Example: find the minimum number of items that together guarantee to cover something
# Definitions

# STATE = contains all the relevant informations to represent the current situation

Tile = set of points ==> it is PROBLEM_SIZE

The problem is to maximize the number of not_taken or otherwise minimize the number of taken.
In other words, covering all points at least once with the least number of tiles possible

In [2]:
from random import random
import numpy as np
from functools import reduce
from collections import namedtuple
from queue import PriorityQueue, SimpleQueue, LifoQueue

In [3]:
PROBLEM_SIZE = 5
NUM_SETS = 10

In [4]:
from typing import Sequence
sets = tuple(np.array([random() < .2 for _ in range(PROBLEM_SIZE)]) for _ in range(NUM_SETS))
State = namedtuple('State', ['taken', 'not_taken'])

PROBLEM_SIZE = 3
NUM_SETS = 5
sets = ([False, False, False],
       [False, False, True],
       [False, True, False],
       [True, False, False],
       [True, True, True])

In [5]:
def goal_check(state):
  #np.all = checks whether all elements are TRUE
  #reduce = applies the logical or to all elements of the specified iterable
  return np.all(reduce(np.logical_or, [sets[i] for i in state.taken], np.array([False for _ in range(PROBLEM_SIZE)])))

In [6]:
assert(goal_check(State(set(range(NUM_SETS)), set())), "Problem not solvable")

<>:1: SyntaxWarning: assertion is always true, perhaps remove parentheses?
<>:1: SyntaxWarning: assertion is always true, perhaps remove parentheses?
<>:1: SyntaxWarning: assertion is always true, perhaps remove parentheses?
C:\Users\Lorenzo\AppData\Local\Temp/ipykernel_14048/987543073.py:1: SyntaxWarning: assertion is always true, perhaps remove parentheses?
  assert(goal_check(State(set(range(NUM_SETS)), set())), "Problem not solvable")


In [7]:
from collections import namedtuple
State = namedtuple('State', ['taken', 'not_taken'])

In [8]:
# A* algorithm implementation

def goal_check(state):
    # This is a binary output: did we reach a solution? True of False
    return np.all(reduce(
        np.logical_or,
        [sets[i] for i in state.taken],
        np.array([False for _ in range(PROBLEM_SIZE)]),
    ))


def h(state):
    # This is a numeric output that counts the amount of TRUEs in the given state. 
    # TRUE means "I am covering that individual point"
    # Since the PriorityQueue is ordered in reverse, we do PROBLEM_SIZE - [...]
    return PROBLEM_SIZE - sum(
        reduce(
            np.logical_or,
            [sets[i] for i in state.taken],
            np.array([False for _ in range(PROBLEM_SIZE)]),
        ))

def g(state):
    return len(state.taken)

def f(state):
    return g(state) + h(state)

def print_sets():
  print("PRINTING SETS:")
  for s in range(0,NUM_SETS):
    print(f"{s}) {sets[s]}")
  print("-"*25)

print_sets()
frontier = PriorityQueue()
state = State(set(), set(range(NUM_SETS)))
frontier.put((f(state), state))

counter = 0
_, current_state = frontier.get()
while not goal_check(current_state):
    counter += 1
    for action in current_state[1]:
        new_state = State(
            current_state.taken ^ {action},
            current_state.not_taken ^ {action},
        )
        print(f"Putting {f(new_state)} with action {new_state}")
        frontier.put((f(new_state), new_state))
    _, current_state = frontier.get()


print(
    f"Solved in {counter:,} steps ({len(current_state.taken)} tiles. Sets used to cover {current_state.taken})"
)

print_sets()

PRINTING SETS:
0) [False, False, False]
1) [False, False, True]
2) [False, True, False]
3) [True, False, False]
4) [True, True, True]
-------------------------
Putting 4 with action State(taken={0}, not_taken={1, 2, 3, 4})
Putting 3 with action State(taken={1}, not_taken={0, 2, 3, 4})
Putting 3 with action State(taken={2}, not_taken={0, 1, 3, 4})
Putting 3 with action State(taken={3}, not_taken={0, 1, 2, 4})
Putting 1 with action State(taken={4}, not_taken={0, 1, 2, 3})
Solved in 1 steps (1 tiles. Sets used to cover {4})
PRINTING SETS:
0) [False, False, False]
1) [False, False, True]
2) [False, True, False]
3) [True, False, False]
4) [True, True, True]
-------------------------
